## Implementing a clasical glacier evolution model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
from scipy import ndimage

In [ ]:

# Physical parameters
Lx = 80000    # Domain length in x (m)
Ly = 85000    # Domain length in y (m)
ttot = 7000   # Time limit (yr)
grad_b = 0.01 # Mass balance gradient (no unit)
b_max = 0.1   # Maximum precip (m/yr)
Z_ELA = 2500  # Elevation of equilibrium line altitude (m)
rho = 910.0   # Ice density (g/m^3)
g   = 9.81    # Earth's gravity (m/s^2)
fd  = 1e-18   # Deformation constant (Pa^-3 y^-1) 


# Numerical parameters
nx = 100  # Number of cells in x
ny = 100  # Number of cells in y


# Initialization & load data
Z_topo = np.loadtxt('mte_rosa.dat')
Z_topo = ndimage.zoom(Z_topo, (ny/Z_topo.shape[0], nx/Z_topo.shape[1]), order=1) 
 
nout = 5000  # Frequency of plotting
dtmax = 1   # maximum time step
dt = dtmax  # Initial time step
dx = Lx / (nx - 1)  # Cell size in x
dy = Ly / (ny - 1)  # Cell size in y
x = np.linspace(0, Lx, nx)  # x-coordinates
y = np.linspace(0, Ly, ny)  # y-coordinates


H_ice = np.zeros((ny, nx))  # Initial ice thickness
Z_surf = Z_topo + H_ice  # Initial ice surface
time = 0  # Initial time
it = 0

In [ ]:

# Loop
while time < ttot:
    
    # Update time
    time += dt
    it   += 1


    # Calculate H_avg, size (ny-1,nx-1)
    H_avg = 0.25 * (H_ice[:-1, :-1] + H_ice[1:, 1:] + H_ice[:-1, 1:] + H_ice[1:, :-1])


    # Compute Snorm, size (ny-1,nx-1)
    Sx = np.diff(Z_surf, axis=1) / dx
    Sy = np.diff(Z_surf, axis=0) / dy
    Sx = 0.5 * (Sx[:-1, :] + Sx[1:, :])
    Sy = 0.5 * (Sy[:, :-1] + Sy[:, 1:])
    Snorm = np.sqrt(Sx**2 + Sy**2)


    # Compute D, size (ny-1,nx-1)
    D = fd * (rho * g)**3.0 * H_avg**5 * Snorm**2


    # Compute dt
    dt = min(min(dx, dy)**2 / (4.1 * np.max(D)), dtmax)


    # Compute qx, size (ny-2,nx-1)
    qx = -(0.5 * (D[:-1,:] + D[1:,:])) * np.diff(Z_surf[1:-1,:], axis=1) / dx


    # Compute qy, size (ny-1,nx-2)
    qy = -(0.5 * (D[:,:-1] + D[:,1:])) * np.diff(Z_surf[:,1:-1,], axis=0) / dy


    # Update rule (diffusion)
    dHdt = -(np.diff(qx, axis=1) / dx + np.diff(qy, axis=0) / dy)
    H_ice[1:-1, 1:-1] += dt * dHdt # size (ny-2,nx-2)


    b = np.minimum(grad_b * (Z_surf - Z_ELA), b_max)


    # Update rule (mass balance)
    H_ice[1:-1, 1:-1] += dt * b[1:-1, 1:-1]


    # Update rule (positive thickness)
    H_ice = np.maximum(H_ice, 0)


    # updatesurface topography
    Z_surf = Z_topo + H_ice
  
    # Change ELA after 5000 years
    if time > 5000:
        Z_ELA = 2900

    # Display
    if it % nout == 0:
        plt.figure(2, figsize=(11, 4),dpi=200)
        plt.subplot(1, 2, 1)
        plt.imshow(Z_surf, extent=[0, Lx/1000, 0, Ly/1000],  cmap='terrain',origin='lower')
        plt.colorbar(label='Elevation (m)')
        plt.title('Ice Surface at ' + str(int(time))+ ' y') 
        plt.xlabel('Distance, km') 
        plt.ylabel('Distance, km') 


        plt.subplot(1, 2, 2)
        plt.imshow(np.where(H_ice>0,H_ice,np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet',origin='lower')
        plt.colorbar(label='Ice Thickness (m)')
        plt.title('Ice Thickness at ' + str(int(time))+ ' y') 
        plt.xlabel('Distance, km') 
        plt.ylabel('Distance, km') 
        plt.show()

